# Trika : The AI Personal Assistant

#### What can this do this so far? (Examples)
* <b>The chatbot is always on. So it is always listening</b>
    * what time is it?
    * what time is it now?
    * what time is it in Sydney?
    * what time is it in Sydney,NS?
    * what is a pigeon? 
        * Pull information from Wikipedia
        * Cannot take double words yet (Ex: what is new delhi?)
    * Can you please add groceries to my to do list
    * add groceries to my to do list
    * what is on my to do list
    * show me what is on my to do list
    * is groceries on my to do list?
        * Yes
        * No, do you want me to add it?
    * Remove groceries from my to do list
    * delete groceries from my to do list
    

#### Import Libraries

In [ ]:
!pip install bs4
!pip install geopy
!pip install tzwhere

In [1]:
from bs4 import BeautifulSoup
import urllib2
import time
from datetime import datetime
from geopy import geocoders
from tzwhere import tzwhere
from pytz import timezone

In [2]:
to_do_list = []

def alwayson():
    for i in range(10,100):
        question = raw_input()
        trika(question)
    
def trika(question):
    Q_Type = ['time']
    Q_Type_List = ['list?','list']
    Q_Type_What = ['What','what']
    Q_Type_Isthere = ['is','IS','there','THERE']
    Q_Type_Remove = ['remove','Remove','Delete','delete']
    list1 = []
    search = ""
    list1 = question.split(" ")
    is_it_a_time_Q = [i for i in list1 if i in Q_Type]
    is_it_a_list_Q = [i for i in list1 if i in Q_Type_List]
    is_it_a_what_Q = [i for i in list1 if i in Q_Type_What]
    is_it_a_isthere_Q = [i for i in list1 if i in Q_Type_Isthere]
    is_it_a_remove_Q = [i for i in list1 if i in Q_Type_Remove]
    #print len(is_it_a_what_Q)
    
    # ***** List of things Trika can do? ***
    # ***** Respond to question (Trika, What can you do?)
    
    #******************************
    #****** Knowledge Q's ********* (Lots to do here)

    if len(is_it_a_time_Q) == 0 and len(is_it_a_list_Q) == 0:
        for i in range(len(list1)):
            if i == (len(list1)-1):
                search =  str(list1[i][:-1])
        link = "https://en.wikipedia.org/wiki/" + search
        #print link
        response = urllib2.urlopen(link)
        html_doc = response.read()
        soup = BeautifulSoup(html_doc, 'html.parser')
        print soup.p.text
        return ""


    #******************************
    #****** Time Q's **************

    if len(is_it_a_time_Q) <> 0:
        for i in range(len(list1)):
            if i == (len(list1)-1):
                search =  str(list1[i][:-1])

    # if the question is "what time is it or what time is it now?"            
        if search == 'it' or search == 'now': 
            time = datetime.now()
            conv_time = str(time)[0:19]
            now_hour = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%I')
            now_minute = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%M')
            now_am_pm = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%p')
            print "[Trika]: " + "It is "+ now_hour + ":" + now_minute + " " + now_am_pm + " now"
            return ""

        # If the question is what time is it somewhere else?

        if search <> 'it':
            locationList =[]
            g = geocoders.GoogleV3()
            #tz = tzwhere.tzwhere()
            locationList.append(search)
            for location in locationList:
                place, (lat, lng) = g.geocode(location)
                tz = str(g.timezone((lat, lng)))
                time = timezone(tz)
                conv_time1 = str(datetime.now(time))[0:10]
                now_dow = datetime.strptime(conv_time1,'%Y-%m-%d').strftime('%A')
                now_month = datetime.strptime(conv_time1,'%Y-%m-%d').strftime('%B')
                now_day = datetime.strptime(conv_time1,'%Y-%m-%d').strftime('%d')
                now_year = datetime.strptime(conv_time1,'%Y-%m-%d').strftime('%Y')

                conv_time = str(datetime.now(time))[0:19]
                now_hour = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%I')
                now_minute = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%M')
                now_am_pm = datetime.strptime(conv_time,'%Y-%m-%d %H:%M:%S').strftime('%p')

                print "[Trika]: " +  "It is "+ now_hour + ":" + now_minute + " " + now_am_pm + " on "+now_dow + ", " + now_month + " " + now_day + "," + now_year + " in " + search
                return ""
        
    #******************************
    #****** To do list ************
    
    
    if len(is_it_a_list_Q) <> 0:
        # find the position of "Add"
        # Add Event
        if len(is_it_a_what_Q) == 0 and len(is_it_a_isthere_Q) == 0 and len(is_it_a_remove_Q) == 0:
            list2 = [x.upper() for x in list1]
            add_pos = [i for i, s in enumerate(list2) if 'ADD' in s]
            len_list2 = len(list2)
            to_do_list.append(str(list2[add_pos[0]:len_list2-5][1]))
            print "[Trika]: " + list2[add_pos[0]:len_list2-5][1] + " has been added to your list"
            return ""

        # Read Event
        if len(is_it_a_what_Q) <> 0:
            print "[Trika]: " + "Here is everything on your list of things to do..."
            print to_do_list
            return ""
        
        # Is it there Event
        if len(is_it_a_isthere_Q) <> 0:
            list2 = [x.upper() for x in list1]          
            if list2[1] in to_do_list:
                print "[Trika]: " + "Yes, " + list2[1] +" is on your list already"
            else:
                print "[Trika]: " + "No, " + list2[1] +" is not on your list, do you want me to do add it?"
                yes_no = raw_input()
                yes_no = yes_no.split(" ")
                if 'NO' == yes_no[0].upper():
                    print "[Trika]: " + "Allright, I will not add it"
                if 'YES' == yes_no[0].upper():
                    to_do_list.append(list2[1])
                    print "[Trika]: Ok, " + list2[1] + " is added to the list"
            return ""
        
        # Remove Event
        if len(is_it_a_remove_Q) <> 0:
            list2 = [x.upper() for x in list1]  
            #to_do_list.remove([x for x in to_do_list if x in list2[1]])
            a = [x for x in to_do_list if x != list2[1]]
            to_do_list[:] = []
            for i in range(len(a)):
                to_do_list.append(a[i])
            print "[Trika]: " + "Removed,here is your new list (Top 10)"
            print to_do_list
            return ""
        
        
        
    
                

In [ ]:
alwayson()